In [ ]:
pip install stop_words nltk pymorphy2 gensim

In [1]:
pip uninstall gensim --yes

Found existing installation: gensim 4.3.1
Uninstalling gensim-4.3.1:
  Successfully uninstalled gensim-4.3.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gensim --no-binary :all:

  Using cached gensim-4.3.1.tar.gz (23.3 MB)
Skipping wheel build for gensim, due to binaries being disabled for it.
    Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install numpy==1.21

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping  

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn import model_selection, preprocessing, linear_model

from gensim.models import Word2Vec
from gensim import corpora
from collections import defaultdict, Counter

In [4]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [6]:
reviews = pd.read_excel('отзывы за лето.xls')

In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [8]:
reviews.dropna(inplace=True)

In [9]:
# reviews['char_count'] = reviews.Content.apply(lambda x: len(str(x)))
# reviews['word_count'] = reviews.Content.apply(lambda x: len(str(x).split()))
# reviews['word_density'] = reviews['char_count'] / (reviews['word_count']+1)
# reviews['punctuation_count'] = reviews.Content.apply(lambda x: len("".join(_ for _ in str(x) if _ in punctuation))) 
# reviews['title_word_count'] = reviews.Content.apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
# reviews['upper_case_word_count'] = reviews.Content.apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [10]:
morph = MorphAnalyzer()

def preprocess_text(txt):
  txt = str(txt)
  txt = txt.lower()
  txt = re.sub('[0-9]|[-—.,:;_%©«»?*!@#№$^•·"&()]|[+=]|[[]|[]]|[/]|[..]|[...]|[....]|[d]', ' ', txt)
  txt = [morph.parse(word)[0].normal_form for word in txt.split() if word not in stopwords.words('russian')]
  # txt = " ".join(txt)
  return txt

In [11]:
reviews['clean_text'] = reviews.Content.apply(preprocess_text)
reviews['label'] = reviews.Rating.map({5:0, 4:0, 3:1, 2:1, 1:1})

<ipython-input-10-9a365ce90dba>:6: FutureWarning: Possible nested set at position 40
  txt = re.sub('[0-9]|[-—.,:;_%©«»?*!@#№$^•·"&()]|[+=]|[[]|[]]|[/]|[..]|[...]|[....]|[d]', ' ', txt)


In [12]:
reviews.tail(7)

,Rating,Content,Date,clean_text,label
20652,2,"Meizu pro6 после обновления пишет, что тел рутованный, хотя рут никто не получал....",2017-06-01,"[meizu, pro, обновление, писать, тело, рутованный, хотя, рута, никто, получать]",1
20653,5,"Легко пользоваться этим приложением, благодаря ему можно сохранить много своего личного времени. Спасибо вам огромное!",2017-06-01,"[легко, пользоваться, это, приложение, благодаря, сохранить, свой, личный, время, спасибо, огромный]",0
20654,1,"Ну и шляпа,с роот правами бесполезная прога,разрабы,вы чего охренеои что ли,зачем в приложении антивирус запихали,у людей свои стоят,испортили приложение👎👎👎",2017-06-01,"[шляпа, роот, право, бесполезный, прога, разраб, охренеой, приложение, антивирус, запихать, человек, свой, стоять, испортить, приложение👎👎👎]",1
20655,5,Ок,2017-06-01,[около],0
20656,4,Доволен,2017-06-01,[довольный],0
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,"[песопаснастя, рута, нужын]",1
20658,5,Сбербанк бомбовая компания на сегодняшний день. Лидер банков.!! Держитесь в том же духе.,2017-06-01,"[сбербанк, бомбовый, компания, сегодняшний, день, лидер, банк, держаться, дух]",0


In [13]:
train_df, valid_df = model_selection.train_test_split(reviews)

In [14]:
full_corpus = []

for text in train_df.clean_text:
  for word in text:
    full_corpus.append(word)

print(len(full_corpus))

87614


In [38]:
freq_dict = Counter(full_corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
len(freq_dict_sorted)

7415

In [16]:
high_freq_words = []
mid_freq_words = []
low_freq_words = []

for word, freq in freq_dict_sorted:
  if freq >= 100:
    high_freq_words.append(word)
  elif freq < 100 and freq >= 10:
    mid_freq_words.append(word)
  else: 
    low_freq_words.append(word)

len(high_freq_words), len(mid_freq_words), len(low_freq_words)

(140, 844, 6431)

In [17]:
vocab = high_freq_words + mid_freq_words
len(vocab)

984

In [18]:
vec1 = CountVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab)
x_train_cv = vec1.fit_transform(train_df.clean_text).toarray()
x_valid_cv = vec1.fit_transform(valid_df.clean_text).toarray()

In [19]:
vec2 = TfidfVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab)
x_train_tfidf = vec2.fit_transform(train_df.clean_text).toarray()
x_valid_tfidf = vec2.fit_transform(valid_df.clean_text).toarray()

In [20]:
max_words = len(freq_dict_sorted)
max_len = len(vocab)

# Training
epochs = 20
batch_size = 128
print_batch_n = 100

In [21]:
x_train_cv.shape, x_train_tfidf.shape 

((15492, 984), (15492, 984))

In [22]:
x_valid_cv.shape, x_valid_tfidf.shape

((5164, 984), (5164, 984))

In [23]:
num_classes = 2
y_train = keras.utils.to_categorical(train_df.label, num_classes)
y_valid = keras.utils.to_categorical(valid_df.label, num_classes)

In [24]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [25]:
model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train_cv, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])


Epoch 1/20
109/109 [==============================] - 63s 113ms/step - loss: 0.5084 - accuracy: 0.8104 - val_loss: 0.4501 - val_accuracy: 0.8232
Epoch 2/20
109/109 [==============================] - 12s 108ms/step - loss: 0.4402 - accuracy: 0.8289 - val_loss: 0.4464 - val_accuracy: 0.8226


In [27]:
score = model.evaluate(x_valid_cv, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 2s 47ms/step - loss: 0.4296 - accuracy: 0.8304


Test score: 0.42962104082107544
Test accuracy: 0.8303640484809875


In [28]:
history = model.fit(x_train_tfidf, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])


Epoch 1/20
109/109 [==============================] - 12s 110ms/step - loss: 0.4763 - accuracy: 0.8092 - val_loss: 0.4673 - val_accuracy: 0.8052
Epoch 2/20
109/109 [==============================] - 12s 108ms/step - loss: 0.4636 - accuracy: 0.8092 - val_loss: 0.4676 - val_accuracy: 0.8052


In [29]:
score = model.evaluate(x_valid_tfidf, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 2s 44ms/step - loss: 0.4599 - accuracy: 0.8116


Test score: 0.4599459171295166
Test accuracy: 0.8115801811218262


In [44]:
import gensim
import zipfile
embed_model = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True)

['xxxxxxxx_NUM',
 'год_NOUN',
 'xxxxxx_NUM',
 'xxxxxxx_NUM',
 'первый_ADJ',
 'время_NOUN',
 'человек_NOUN',
 'introduction_PROPN',
 'один_NUM',
 'район_NOUN',
 'мочь_VERB',
 'стать_VERB',
 'так_ADV',
 'город_NOUN',
 'другой_ADJ',
 'область_NOUN',
 'быть_VERB',
 'новый_ADJ',
 'два_NUM',
 'перенаправление_NOUN',
 'состав_NOUN',
 'часть_NOUN',
 'россия_PROPN',
 'место_NOUN',
 'также_ADV',
 'являться_VERB',
 'где_ADV',
 'иметь_VERB',
 'работа_NOUN',
 'война_NOUN',
 'самый_ADJ',
 'уже_ADV',
 'получить_VERB',
 'второй_ADJ',
 'день_NOUN',
 'век_NOUN',
 'еще_ADV',
 'российский_ADJ',
 'группа_NOUN',
 'жизнь_NOUN',
 'имя_NOUN',
 'мир_NOUN',
 'игра_NOUN',
 'река_NOUN',
 'более_ADV',
 'сказать_VERB',
 'русский_ADJ',
 'говорить_VERB',
 'находиться_VERB',
 'советский_ADJ',
 'государственный_ADJ',
 'дело_NOUN',
 'сам_ADJ',
 'большой_ADJ',
 'фильм_NOUN',
 'делать_VERB',
 'вид_NOUN',
 'километр_NOUN',
 'сша_PROPN',
 'население_NOUN',
 'работать_VERB',
 'известный_ADJ',
 'деревня_NOUN',
 'однако_ADV',
 

In [49]:
def gensim_to_keras_embedding(model, train_embeddings=False, remove=[]):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model   # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    remove_indices = []
    for word in remove:
        index = model.get_index(word)
        if index == None:
            continue
        remove_indices.append(index)
    weights = np.delete(weights, remove_indices, axis=0)
    #index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [47]:
#gensim_to_keras_embedding(embed_model)
vocab_pretrained = []
counter = 0
seen_words = set()
words_to_remove = []
for k, v in embed_model.key_to_index.items():
  und_ind = k.index('_')
  mod_k = k[0:und_ind]

  if mod_k not in freq_dict:
    words_to_remove.append(k)
    continue
    
  if mod_k in seen_words:
    words_to_remove.append(k)
    continue
    #mod_k = mod_k+str(counter)
    #counter += 1

  vocab_pretrained.append(mod_k)
  seen_words.add(mod_k)
  

print(len(vocab_pretrained))

5079


In [40]:
vec1 = CountVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab_pretrained, max_features=300)
x_train_cv = vec1.fit_transform(train_df.clean_text).toarray()
x_valid_cv = vec1.fit_transform(valid_df.clean_text).toarray()

In [41]:
vec2 = TfidfVectorizer(lowercase=False, analyzer=lambda x: x, ngram_range=(1, 4), vocabulary=vocab_pretrained, max_features=300)
x_train_tfidf = vec2.fit_transform(train_df.clean_text).toarray()
x_valid_tfidf = vec2.fit_transform(valid_df.clean_text).toarray()

In [42]:
max_words = len(vocab_pretrained)
max_len = 300

# Training
epochs = 20
batch_size = 128
print_batch_n = 100

In [43]:
x_train_cv.shape, x_train_tfidf.shape 

((15492, 5079), (15492, 5079))

In [50]:
model = Sequential()
model.add(gensim_to_keras_embedding(embed_model, remove=words_to_remove))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [51]:
model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [52]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train_cv, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])


Epoch 1/20
109/109 [==============================] - 63s 559ms/step - loss: 0.4789 - accuracy: 0.8073 - val_loss: 0.4465 - val_accuracy: 0.8219
Epoch 2/20
109/109 [==============================] - 60s 547ms/step - loss: 0.4504 - accuracy: 0.8264 - val_loss: 0.4540 - val_accuracy: 0.8258


In [53]:
score = model.evaluate(x_valid_cv, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 10s 242ms/step - loss: 0.4340 - accuracy: 0.8309


Test score: 0.43400633335113525
Test accuracy: 0.8309450149536133


In [54]:
history = model.fit(x_train_tfidf, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])


Epoch 1/20
109/109 [==============================] - 60s 549ms/step - loss: 0.4832 - accuracy: 0.8092 - val_loss: 0.4666 - val_accuracy: 0.8052
Epoch 2/20
109/109 [==============================] - 59s 546ms/step - loss: 0.4673 - accuracy: 0.8092 - val_loss: 0.4715 - val_accuracy: 0.8052


In [55]:
score = model.evaluate(x_valid_tfidf, y_valid, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

41/41 [==============================] - 10s 233ms/step - loss: 0.4660 - accuracy: 0.8116


Test score: 0.46598175168037415
Test accuracy: 0.8115801811218262


Вывод:
Разницы в результатах при использовании эмбеддинга со случайными весами и с предобученными весами не выявлено. Предположительно, если бы было необходимо использовать весб набор слов из предобученной модели, сэкономили бы время на тренировке, в текущих условиях словарь получился довоьно маленький, и разница не заметна.